In [ ]:
# Allows the modification of the files imported without having to restart kernel or re-import them
%load_ext autoreload
%autoreload 2

In [ ]:
##### %matplotlib notebook
# %matplotlib inline
# to have interactive plots : pip install ipympl
%matplotlib ipympl
from matplotlib import rc
rc('figure',figsize=(16,4))
rc('font',size=12)
rc('text',usetex=False)
rc('image', cmap='viridis')

from qubicpack import qubicpack as qp
from qubicpack.utilities import Qubic_DataDir
import qubic.lib.Calibration.Qfiber as ft
import qubic.lib.Fitting.QsynthesizedBeam as sbfit

from pysimulators import FitsArray

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import pickle
import scipy.signal as scsig
from scipy import interpolate
import os
import healpy as hp
import pandas as pd
import fitsio


In [ ]:
peak_file = "/Users/huchet/qubic/qubic/calfiles/fitted_peaks.fits"
data, header = fitsio.read(peak_file, header=True)

In [ ]:
fits=fitsio.FITS(peak_file)

In [ ]:
print(fits)

print(fits[0])

In [ ]:
os.uname()[4]

In [ ]:
#Set paths
machine = os.uname()[1]
def check_dir(dirfile):
    if os.path.isdir( dirfile): 
        print('dirfile:', dirfile); 
    else: 
        raise ValueError('dirfile {} is not an existing directory'.format(dirfile) )
        
if (machine=='apcdhcp52.in2p3.fr') or (machine=='MacJCH-2.local'):
    dirfiles = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
    check_dir(dirfiles)
elif machine == 'shaka':
    dirfiles = "/home/mgamboa/qubic/QUBIC/qubiccalibration/Data/150GHz-2019-04-06/"
    check_dir(dirfiles)
elif machine == 'hain':
    dirfiles = "/home/martin/QUBIC/qubiccalibration/Data/Calib-TD/150GHz-2019-04-06/"
    check_dir(dirfiles)
elif machine in ["apcdhcp130.in2p3.fr", "pc-607.home", "apcdhcp23.in2p3.fr"]: # why different names for my computer? it is the name of the machine on the network
    dirfiles_ = "/Users/huchet/Documents/code/data/ComissioningTD/calib_lab/Synthesized_Beams_Files/"
else:
    raise ValueError("*dirfile variable not defined*. You are in machine named: {}. \
    You have to specify a directory where the synthesized beam maps are saved in \
    Flat or Healpix format (directories)".format(machine))

In [ ]:
dirfiles = dirfiles_ + "130GHz-2019-04-18/"

# Get the data
freq_source = int(dirfiles.split("GHz")[0][-3:]) #130 

# dirfreq = '150GHz-2019-04-06/'
# dirallfreqs = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
# dirfiles = dirallfreqs + dirfreq

c50 = np.cos(np.radians(50))
azmin = -15./c50
azmax = 15./c50


TESNum = 93
# reload(sbfit)
flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, azmin = azmin, azmax=azmax)
npix = len(np.ravel(flatmap))

plt.subplot(1,2,1)
plt.imshow(flatmap,
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], aspect='equal',
      vmin=-3000, vmax=3000)
# plt.colorbar()
plt.show()

In [ ]:
alldata_renorm = np.zeros((256, npix))
for i in range(256):
    flatmap, az, el = sbfit.get_flatmap(i+1, dirfiles, azmin=azmin, azmax=azmax)
    alldata_renorm[i, :] = np.ravel((flatmap - np.mean(flatmap)) / np.std(flatmap))  

In [ ]:
##### Estimating the median of all SB images - to be removed - Origin still unexplained...
med = np.reshape(np.median(alldata_renorm,axis=0),np.shape(flatmap))
mm = np.mean(med)
ss = np.std(med)

plt.imshow(np.reshape(med, np.shape(flatmap)),
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
       aspect='equal')
# plt.colorbar()
plt.title('Median over all S.B. images')
plt.show()

In [ ]:
# reload(sbfit)
TESNum = 93
flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, remove=med, azmin=azmin, azmax=azmax)
# flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles)

mm, ss = ft.meancut(flatmap, 3)
plt.subplot(1,2,1)
plt.imshow(flatmap,
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
       aspect='equal',
       vmin=mm-ss, 
       vmax=mm+10*ss)
# plt.colorbar()
plt.title('TES #{}'.format(TESNum))
plt.show()

In [ ]:
# %%script echo skipping

from matplotlib.widgets import Button

# fig, axs = plt.subplots(1, 4, figsize=(10, 3), width_ratios=(1, 1, 1, 0.05))
fig, axs = plt.subplots(1, 2, figsize=(7, 7), width_ratios=(1, 0.05))

# xs = 201
# reso = 6

class Index:

    def __init__(self, TES_num, fig, axes, prev_selec, verbose=False):
        self.fig = fig
        self.prev_selec = prev_selec
        self.isok_arr = np.zeros(len(TES_num), dtype=bool)
        self.ind = -1
        self.axs = axes
        self.verbose = verbose
        self.goNext()

    
    def test_ind(self):
        if self.ind >= len(self.isok_arr):
            if self.verbose:
                print("Not good index")
            return False
        elif self.prev_selec[self.ind] == False:
            if self.verbose:
                print("Already rejected TES")
            self.isok_arr[self.ind] = False
            self.goNext()
        else:
            return True
    
    def goNext(self):
        self.ind += 1
        if self.verbose:
            print(self.ind)
        if not self.test_ind():
            return
        if self.verbose:
            print("TES {}".format(self.ind + 1))
        flatmap, az, el = sbfit.get_flatmap(self.ind + 1, dirfiles, remove=med, azmin=azmin, azmax=azmax)
        # print("ok", flush=True)
        ax = axs[0]
        ax.clear()
        im = ax.imshow(flatmap,
            extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
            aspect='equal',
            # vmin=mm-ss, 
            # vmax=mm+10*ss
        )
        cax = axs[-1]
        cax.clear()
        plt.colorbar(im, cax=cax)
        plt.subplots_adjust(hspace=0)
        self.fig.suptitle("TES {}".format(self.ind + 1))
        plt.draw()

    def good(self, event):
        if not self.test_ind():
            return
        self.isok_arr[self.ind] = True
        self.goNext()


    def bad(self, event):
        if not self.test_ind():
            return
        self.isok_arr[self.ind] = False
        self.goNext()


allTESNum = np.arange(256) + 1
test_ifok = np.ones_like(allTESNum, dtype=bool)
# callback = Index(allTESNum, fig, axs, visibly_ok_arr, verbose=False)
callback = Index(allTESNum, fig, axs, np.full_like(allTESNum, True), verbose=False)
axbad = fig.add_axes([0.7, 0.05, 0.1, 0.075])
axgood = fig.add_axes([0.81, 0.05, 0.1, 0.075])
bgood = Button(axgood, 'Good')
bgood.on_clicked(callback.good)
bbad = Button(axbad, 'Bad')
bbad.on_clicked(callback.bad)

plt.show()

In [ ]:
%%script echo skipping
# reload(sbfit)
for TESNum in range(1, 257):
    print(TESNum)
    flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, remove=med, azmin=azmin, azmax=azmax)
    # plt.figure()
    # plt.subplot(1,2,1)
    # plt.imshow(flatmap,
    #     extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
    #     aspect='equal',
    #     # vmin=mm-ss, 
    # )
    # # plt.colorbar()
    # plt.title('TES #{}'.format(TESNum))
    # plt.show()
    
    #### Instantiate a model
    sbfitmodel3 = sbfit.SbModelIndepPeaks(nrings=2, common_fwhm=True, no_xy_shift=False, distortion=False)
    
    fit, xypeaks = sbfit.fit_sb(flatmap, az, el, sbfitmodel3, verbose=False, resample=False, 
                            doplot=False, extra_title='TES #{}'.format(TESNum))
    thefile = open(dirfiles + '/FitSB/fit-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/fit-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(fit[1], thefile)
    thefile.close
    
    thefile = open(dirfiles + '/FitSB/fiterr-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/fiterr-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(fit[2], thefile)
    thefile.close
    
    thefile = open(dirfiles +'/FitSB/xypeaks-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/xypeaks-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(xypeaks, thefile)
    thefile.close
    # zar


In [ ]:
fit_pos = np.load(dirfiles_ + "Measured_beam_zeroth_peak.npy")

In [ ]:
print(fit_pos.shape)

In [ ]:
print(fit_pos[:, 0])

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
# reload(sbfit)
nperpage = 7
nsiglo = 1.
nsighi = 3.
TESmax = 256
fitted_directory = dirfiles +'/FitSB/'
# fitted_directory = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/'

with PdfPages(dirfiles + '/allFitSB_{}.pdf'.format(freq_source)) as pdf:
# with PdfPages('/Users/hamilton/Google Drive/QUBIC/Calib-TD/allFitSB.pdf') as pdf:
    for TESNum in range(1, TESmax+1):
        ipage = (TESNum-1) % nperpage
        print(TESNum, ipage)
        if ipage  == 0:
            rc('figure', figsize=(20,28))
        
        ### Read map
        flatmap, az, el, themap, newxxyy = sbfit.get_flatmap(TESNum, dirfiles, 
                                                             remove=med, azmin=azmin, azmax=azmax,
                                                             fitted_directory = fitted_directory)
        mm, ss = ft.meancut(flatmap-themap,3)
        
        subplot(nperpage, 4, 1+ipage*4)
        imshow(flatmap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=np.min(flatmap)/10, 
               vmax=np.max(flatmap)/2)
        colorbar()
        title('Input Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 2+ipage*4)
        imshow(flatmap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        sh = newxxyy.shape
        for i in range(sh[1]):
            ax = plot(newxxyy[0,i], newxxyy[1,i], 'r.')

        colorbar()
        title('Input Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 3+ipage*4)
        imshow(themap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        colorbar()
        title('Fitted Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 4+ipage*4)
        imshow(flatmap-themap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        colorbar()
        title('Residuals Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        if (ipage == (nperpage-1)) or TESNum==TESmax:
            tight_layout()
            pdf.savefig()
            clf()
   
    
    
    
